In [ ]:
!pip install transformers sentencepiece

     |████████████████████████████████| 2.5MB 8.6MB/s 
     |████████████████████████████████| 1.2MB 38.1MB/s 
     |████████████████████████████████| 901kB 36.8MB/s 
     |████████████████████████████████| 3.3MB 40.0MB/s 


In [ ]:
!nvidia-smi

Thu Jul  8 21:00:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P8    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!rm -r results

In [ ]:
!wget http://ciir.cs.umass.edu/downloads/qulac/data/qulac/qulac.json

--2021-07-08 19:26:07--  http://ciir.cs.umass.edu/downloads/qulac/data/qulac/qulac.json
Resolving ciir.cs.umass.edu (ciir.cs.umass.edu)... 128.119.246.154
Connecting to ciir.cs.umass.edu (ciir.cs.umass.edu)|128.119.246.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3799709 (3.6M) [application/json]
Saving to: ‘qulac.json.3’

qulac.json.3        100%[===================>]   3.62M  5.82MB/s    in 0.6s    

2021-07-08 19:26:08 (5.82 MB/s) - ‘qulac.json.3’ saved [3799709/3799709]



In [ ]:
import torch
import shutil
import random
import numpy as np
import transformers
import pandas as pd
import seaborn as sns

from torch import nn
from sklearn.metrics import balanced_accuracy_score, accuracy_score, r2_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedStratifiedKFold
from transformers import AutoModelForSequenceClassification, AutoModel, AutoTokenizer, Trainer, TrainingArguments, EvalPrediction, RobertaTokenizer, RobertaForSequenceClassification

In [ ]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
training = pd.read_csv("training.tsv", sep="\t").drop_duplicates()
training["label"] = 1

# Reset Index & References
training.to_csv("training.csv", index=False)
training = pd.read_csv("training.csv")

t = training
for query in training["query"]:
  question_candidates = np.random.choice(training[training["query"] != query]["clarifying_question"], size = 8) # samples per question
  for question in question_candidates:
    t = t.append({
        "query": str(query),
        "clarifying_question": str(question),
        "label": -1
    }, ignore_index = True)

training = t

training.tail()

,query,clarifying_question,label
27733,i'm interested in history of the electronic me...,would you like to learn about rock and gem sho...,-1
27734,i'm interested in history of the electronic me...,do you need a list of civil war battles,-1
27735,i'm interested in history of the electronic me...,are you interested in heavy trucks part,-1
27736,i'm interested in history of the electronic me...,do you need information on best climate for sp...,-1
27737,i'm interested in history of the electronic me...,in which state are you looking for east ridge ...,-1


roberta large - no qlac - 3 Epochs - R2 0.746606

In [ ]:
PRETRAINED_MODEL = "roberta-large"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL, use_fast=True)
training_encodings = tokenizer(
    list(training["query"]),
    list(training["clarifying_question"]), 
    truncation=True, 
    padding=True
)
training_labels = training["label"]

In [ ]:
class DlnlpDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {}
        for key, val in self.encodings.items():
          item[key] = torch.tensor(val[idx])
        
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
dataset = DlnlpDataset(training_encodings, training_labels)

In [ ]:
train_dataset, validation_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * .8) + 1, int(len(dataset) * .2)])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=1)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

In [ ]:
def f1_eval(p: EvalPrediction):
    predictions = p.predictions
    label_ids = p.label_ids
    return {
        "r2": r2_score(label_ids, predictions),
        "accuracy": accuracy_score(np.clip(label_ids, 0, 1), np.clip(np.round(predictions), 0, 1)),
        "balanced_accuracy": balanced_accuracy_score(np.clip(label_ids, 0, 1), np.clip(np.round(predictions), 0 , 1)),
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        overwrite_output_dir=True,
        output_dir='./results',
        save_strategy="no",
        num_train_epochs=5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=16,
        warmup_steps=40,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        learning_rate=1e-05,
        fp16=True,
    ),
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=f1_eval
)

Using amp fp16 backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 22191
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 1730


Epoch,Training Loss,Validation Loss,R2,Accuracy,Balanced Accuracy
0,0.100700,0.095741,0.756154,0.973860,0.919467


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
***** Running Evaluation *****
  Num examples = 5547
  Batch size = 4


Epoch,Training Loss,Validation Loss,R2,Accuracy,Balanced Accuracy
0,0.100700,0.095741,0.756154,0.973860,0.919467
1,0.093400,0.077925,0.801530,0.976564,0.910967
2,0.079400,0.087595,0.776900,0.976023,0.922830
3,0.064200,0.085892,0.781238,0.976744,0.935401
4,0.022600,0.085768,0.781554,0.977285,0.926402


***** Running Evaluation *****
  Num examples = 5547
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5547
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5547
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5547
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1730, training_loss=0.11070799322831149, metrics={'train_runtime': 3567.7961, 'train_samples_per_second': 31.099, 'train_steps_per_second': 0.485, 'total_flos': 1.2533127989504136e+16, 'train_loss': 0.11070799322831149, 'epoch': 5.0})

In [ ]:
test_set = pd.read_csv("test_set.tsv", sep="\t", names=["query"])
test_set.head(10)

,query
0,Tell me about Computers
1,What is a function
2,Im interested in the current cancellor of germany
3,Im looking for information about alcohol
4,I want to know how long an egg needs to boil
5,I want to know how an engine works
6,How do i train a neural network
7,Tell me about job opportunites at the technisc...
8,How to become an amazon turkey worker
9,Who is the richest man on the planet


In [ ]:
question_bank = pd.read_csv("question_bank.tsv", sep="\t")
question_bank.head(10)

,question_id,question
0,Q00002,a total cholesterol of 180 to 200 mgdl 10 to 1...
1,Q00003,about how many years experience do you want th...
2,Q00004,according to anima the bible or what other source
3,Q00005,ae you looking for examples of septic system d...
4,Q00006,any specific company on your mind
5,Q00007,any tutorials would interest you
6,Q00008,are asking what the symptoms are for angular c...
7,Q00009,are interested in the nightlife here
8,Q00010,are looking for reviews on volvo xc90 suv
9,Q00011,are ozzie guillen and fidel castro related


In [ ]:
queries = []
questions = []
question_ids = []
for question, question_id in zip(question_bank["question"], question_bank["question_id"]):
  for query in test_set["query"]:
    queries.append(query)
    questions.append(question)
    question_ids.append(question_id)

test = pd.DataFrame({
    "query": queries,
    "clarifying_question": questions,
    "question_id": question_ids
})
test.head(10)

,query,clarifying_question,question_id
0,Tell me about Computers,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
1,What is a function,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
2,Im interested in the current cancellor of germany,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
3,Im looking for information about alcohol,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
4,I want to know how long an egg needs to boil,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
5,I want to know how an engine works,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
6,How do i train a neural network,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
7,Tell me about job opportunites at the technisc...,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
8,How to become an amazon turkey worker,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002
9,Who is the richest man on the planet,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002


In [ ]:
test_encodings = tokenizer(
    list(test["query"]), 
    list(test["clarifying_question"]), 
  truncation=True, 
    padding=True
)
test_labels = list(map(int, np.zeros(len(test["query"]))))

In [ ]:
test_dataset = DlnlpDataset(test_encodings, test_labels)

In [ ]:
test["prediction"] = trainer.predict(test_dataset).predictions.reshape(-1)

***** Running Prediction *****
  Num examples = 182160
  Batch size = 4


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [ ]:
test.head(10)

,query,clarifying_question,question_id,prediction
0,Tell me about Computers,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.989867
1,What is a function,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.980278
2,Im interested in the current cancellor of germany,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.998335
3,Im looking for information about alcohol,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.986084
4,I want to know how long an egg needs to boil,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-1.003779
5,I want to know how an engine works,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.996259
6,How do i train a neural network,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.993569
7,Tell me about job opportunites at the technisc...,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.985211
8,How to become an amazon turkey worker,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.998608
9,Who is the richest man on the planet,a total cholesterol of 180 to 200 mgdl 10 to 1...,Q00002,-0.990558


In [ ]:
test = test.sort_values(["query", "prediction"], ascending=False)
test.head(10)

,query,clarifying_question,question_id,prediction
152713,whats inside a calzone,would you like to buy tickets to california,Q03321,1.114958
120513,whats inside a calzone,is there anywhere in california youd like to g...,Q02621,1.111786
73915,whats inside a calzone,do you need anything specific in relation to c...,Q01608,1.096509
54871,whats inside a calzone,are you planning to take a trip on ontario or ...,Q01194,1.079110
30307,whats inside a calzone,are you looking for a recipe,Q00660,1.076265
139327,whats inside a calzone,which ingredients do you want to include or ex...,Q03030,1.070570
171895,whats inside a calzone,would you like to learn about rock and gem sho...,Q03738,1.065127
41761,whats inside a calzone,are you looking for information on weather,Q00909,1.053256
24419,whats inside a calzone,are you interested in visiting ontario califor...,Q00532,1.025422
55147,whats inside a calzone,are you referencing earthquake activity,Q01200,1.015179


In [ ]:
out = test.groupby([ "query" ]).head(50).reset_index(drop=True)
out

,query,clarifying_question,question_id,prediction
0,whats inside a calzone,would you like to buy tickets to california,Q03321,1.114958
1,whats inside a calzone,is there anywhere in california youd like to g...,Q02621,1.111786
2,whats inside a calzone,do you need anything specific in relation to c...,Q01608,1.096509
3,whats inside a calzone,are you planning to take a trip on ontario or ...,Q01194,1.079110
4,whats inside a calzone,are you looking for a recipe,Q00660,1.076265
...,...,...,...,...
2295,How do i plug a mouse into the computer,are you looking for a list of products,Q00639,-0.783600
2296,How do i plug a mouse into the computer,are you looking for the cost,Q01060,-0.787121
2297,How do i plug a mouse into the computer,are you looking for a brooks brothers outlet,Q00594,-0.796877
2298,How do i plug a mouse into the computer,would you like to see reviews by professionals...,Q03878,-0.797915


In [ ]:
out["question_ids"] = list(out.groupby(["query"])["question_id"].transform(lambda x: ','.join(x)))
out

,query,clarifying_question,question_id,prediction,question_ids
0,whats inside a calzone,would you like to buy tickets to california,Q03321,1.114958,"Q03321,Q02621,Q01608,Q01194,Q00660,Q03030,Q037..."
1,whats inside a calzone,is there anywhere in california youd like to g...,Q02621,1.111786,"Q03321,Q02621,Q01608,Q01194,Q00660,Q03030,Q037..."
2,whats inside a calzone,do you need anything specific in relation to c...,Q01608,1.096509,"Q03321,Q02621,Q01608,Q01194,Q00660,Q03030,Q037..."
3,whats inside a calzone,are you planning to take a trip on ontario or ...,Q01194,1.079110,"Q03321,Q02621,Q01608,Q01194,Q00660,Q03030,Q037..."
4,whats inside a calzone,are you looking for a recipe,Q00660,1.076265,"Q03321,Q02621,Q01608,Q01194,Q00660,Q03030,Q037..."
...,...,...,...,...,...
2295,How do i plug a mouse into the computer,are you looking for a list of products,Q00639,-0.783600,"Q01567,Q01058,Q03942,Q01037,Q01867,Q00709,Q029..."
2296,How do i plug a mouse into the computer,are you looking for the cost,Q01060,-0.787121,"Q01567,Q01058,Q03942,Q01037,Q01867,Q00709,Q029..."
2297,How do i plug a mouse into the computer,are you looking for a brooks brothers outlet,Q00594,-0.796877,"Q01567,Q01058,Q03942,Q01037,Q01867,Q00709,Q029..."
2298,How do i plug a mouse into the computer,would you like to see reviews by professionals...,Q03878,-0.797915,"Q01567,Q01058,Q03942,Q01037,Q01867,Q00709,Q029..."


In [ ]:
out[["query", "question_ids"]].drop_duplicates().to_csv("answer.txt", sep="\t", header=False, index=False)